In [ ]:
# 從google.colab庫中導入drive模組，連線到google drive
# 將google drive掛載到colab環境上，並將掛載路徑設置為'/content/gdrive/'
# 設定force_remount=True以強制重新掛載，以防google drive已經掛載
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

# 導入os模組以與操作系統進行互動
# 將當前工作目錄更改為google drive中指定的路徑
# 執行'ls'，列出指定目錄中的文件和目錄
import os
os.chdir("/content/gdrive/My Drive/112_2_LATIA/hw2_0507")
os.listdir()

Mounted at /content/gdrive/


['dune_series_book_info.csv', 'data_analysis_intern_info.csv']

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.ehGPLqUsnV/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.k902kMj40M/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.kHiQhyroH9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [ ]:
!pip install selenium

In [ ]:
# 導入Selenium的webdriver模組
# 從Selenium函式庫的common.by模組中導入By
from selenium import webdriver
from selenium.webdriver.common.by import By

# 建立ChromeOptions物件
# 使用add_argument()方法添加--headless參數，使Chrome以headless模式運行，即在後台運行，不顯示GUI界面
# 使用add_argument()方法添加--no-sandbox參數，這是一個必要的參數，用於解決在某些環境下的運行問題
# 將 headless 屬性設置為 True
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True


# 初始化Chrome WebDriver，並傳遞Chromedriver的路徑和選項
# 設置隱式等待時間為10秒。隱式等待是在查找元素時的一種等待方式，如果WebDriver沒有找到元素，它會等待一定的時間再進行查找，直到超過設定的時間為止
driver=webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)

# 使用get方法訪問指定的URL，在此是'https://www.104.com.tw/jobs/main/students/?tab=intern&ro=2&pagesize=50&jobsource=students_intern&showDutyTime=1&hotJob=0&rostatus=1024&keyword=%E5%AF%A6%E7%BF%92&order=12&asc=0'
# 使用Selenium的定位函數，使用find_element()方法透過 CSS 選擇器 (CSS_SELECTOR) 根據元素的樣式、類別、屬性等特徵來查找指定的元素，在此是'div[class='students-list-item position-relative']'，並存到jobs列表中
driver.get("https://www.104.com.tw/jobs/main/students/?tab=intern&ro=2&pagesize=50&jobsource=students_intern&showDutyTime=1&hotJob=0&rostatus=1024&keyword=%E5%AF%A6%E7%BF%92&order=12&asc=0")
jobs=driver.find_elements(By.CSS_SELECTOR,"div[class='students-list-item position-relative']")

# 初始化空列表jobs_info以存儲工作資料
# 對jobs列表中job工作資料進行迴圈
# 使用Selenium的定位函數，分別找出每個工作資料對應的職位、公司、薪資、上班時段、工作地點、刊登日期和連結
# 將找出的工作資料以字典的形式加到jobs_info列表中
jobs_info=[]
for job in jobs:

  job_name=job.find_element(By.TAG_NAME,"a").text
  job_company=job.find_element(By.CSS_SELECTOR,"p[class='h4 text-gray-deep-dark text-truncate mb-1'").text
  job_pay=job.find_element(By.CSS_SELECTOR,"span[class='text-primary h4'").text
  job_time=job.find_element(By.CSS_SELECTOR,"span[class='t4 text-gray-deep-dark'").text
  job_city=job.find_element(By.CSS_SELECTOR,"p[class='job-otherDescription text-gray-darker t5 mb-0 pt-1'").text.split("，")[0]
  job_date=job.find_element(By.CSS_SELECTOR,"p[class='job-otherDescription text-gray-darker t5 mb-0 pt-1'").text.split("，")[1]
  job_url=job.find_element(By.TAG_NAME,"a").get_attribute("href")
  jobs_info.append({"職位":job_name,"公司":job_company,"薪資":job_pay,"上班時段":job_time,"工作地點":job_city,"刊登日期":job_date,"連結":job_url})

# 對jobs列表中job工作資料進行迴圈，印出每個工作資料
for job_info in jobs_info:
  print(job_info)

# 這個方法會關閉WebDriver並結束瀏覽器進程。這是為了確保在使用完瀏覽器後，程式可以正確地退出
driver.quit()


{'職位': '【高雄學生實習Internship】 (2023年度在校生) 工程部門實習生', '公司': '富台工程股份有限公司', '薪資': '時薪183~190元', '上班時段': '日班，8:00-17:00', '工作地點': '高雄市三民區', '刊登日期': '5/06更新', '連結': 'https://www.104.com.tw/job/80sa0'}
{'職位': '企業實習生-學生實習', '公司': '宏得國際企業有限公司', '薪資': '時薪183元以上', '上班時段': '日班/假日班，09:00-18:00，需輪班', '工作地點': '台中市龍井區', '刊登日期': '4/14更新', '連結': 'https://www.104.com.tw/job/6v2tt'}
{'職位': '【長期實習】市場研究實習生', '公司': '德商GfK_捷孚凱行銷研究顧問有限公司', '薪資': '時薪183元', '上班時段': '日班，09:00-18:00 (可根據自己的課表彈性在區間內安排上班)', '工作地點': '新北市汐止區', '刊登日期': '4/30更新', '連結': 'https://www.104.com.tw/job/3oo3w'}
{'職位': '25 資訊實習生', '公司': '新代科技股份有限公司', '薪資': '時薪210~265元', '上班時段': '日班，08:30~17:30', '工作地點': '新竹市', '刊登日期': '5/03更新', '連結': 'https://www.104.com.tw/job/6zveq'}
{'職位': '大四實習生(實習期間：暑期113/07/01-學期114/06/30)', '公司': '聯聚建設股份有限公司', '薪資': '月薪33,000元', '上班時段': '日班，08:00-17:30', '工作地點': '台中市西屯區', '刊登日期': '4/30更新', '連結': 'https://www.104.com.tw/job/7qw64'}
{'職位': '【一年期實習】2024 Internship Program - Operation Intern', '公司': '美商億業有限公司台灣分公司', '薪資': '時薪20

In [ ]:
# 導入csv模組
import csv

# 指定csv檔案的路徑，設定為'/content/gdrive/My Drive/112_2_LATIA/hw2_0507/data_analysis_intern_info.csv'
# 開啟csv檔案，使用'w'模式寫入，並設定newline=''避免插入空行
# 定義csv欄位名稱
# 建立csv寫入器
# 寫入csv檔案的標頭行
# 寫入所有工作資訊到csv檔案中
csv_file = './data_analysis_intern_info.csv'
with open(csv_file, mode='w', newline='') as fp:
    fieldnames = ['職位', '公司', '薪資', '上班時段', '工作地點', '刊登日期', "連結"]
    writer = csv.DictWriter(fp, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(jobs_info)
